# Código 13: Malhas e Texturas

## Código pré loop principal

### Inicialização do glfw e criação da janela

In [318]:
#Bibliotecas
try:
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import random
    import glm
    from PIL import Image
except ImportError:
    !pip install glfw
    !pip install pyopengl
    !pip install numpy
    !pip install glm
    !pip install pillow
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import random
    import glm
    from PIL import Image

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

#get_dim_pos: retorna tamanho da tela e posição da tela
def get_dim_pos(per_width = 0.6, per_height = 0.6): 
    # Obtendo configurações do monitor
    monitores = glfw.get_monitors()
    monitor = monitores[0]
    video_mode = glfw.get_video_mode(monitor)
    WIDTH_WINDOW, HEIGHT_WINDOW = video_mode.size
    # Definindo proporção que se quer do monitor
    WIDTH_WINDOW : int = int(per_width*WIDTH_WINDOW)
    HEIGHT_WINDOW : int = int(per_height*HEIGHT_WINDOW)
    POSX_WINDOW : int = (video_mode.size[0] - WIDTH_WINDOW) // 2
    POSY_WINDOW : int = (video_mode.size[1] - HEIGHT_WINDOW) // 2
    return WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW

# Pega tamanho da tela e posição da tela
WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW = get_dim_pos(0.6,0.6)
# Criando janela
TITLE: str = "Exercício de Malhas"
window = glfw.create_window(WIDTH_WINDOW, HEIGHT_WINDOW, TITLE, None, None)
glfw.set_window_pos(window, POSX_WINDOW, POSY_WINDOW)
glfw.make_context_current(window)

### Shaders: Vertex e Fragment

In [319]:
#GLSL para Vertex Shader
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_rot_x;
        uniform mat4 mat_rot_y;
        uniform mat4 mat_rot_z; 
        uniform mat4 mat_scale;  
        uniform mat4 mat_transl; 

        attribute vec2 texture_coord;
        varying vec2 out_texture;

        void main(){
            gl_Position = mat_transl * mat_scale * mat_rot_x * mat_rot_y * mat_rot_z * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

#GLSL para Fragment Shader
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Solicitando espaço, compilando e linkando

In [320]:
#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

### Definindo classes para objetos do cenário

criar um angulo para cada eixo para poder movimentar cada um dos eixos

In [321]:
# Classe para criação de elementos que se movimentam no cenário
class graphic_element:
    def __init__(   self, 
                    inicial_vert = 0, num_vert = 0, 
                    pos_x = 0, pos_y = 0, pos_z = 0,
                    angle = 0,
                    scale = 1,
                    linear_speed = 0.05, angular_speed = 0.139, scale_speed = 0.2,
                    limit_sup = 1,
                    limit_inf = -1):
        # Atributos de identificação
        self._inicial_vert = inicial_vert
        self._num_vert = num_vert
        # Atributos de posição
        self._pos_x = pos_x
        self._pos_y = pos_y
        self._pos_z = pos_z
        self._linear_speed = linear_speed
        self._limit_sup = limit_sup
        self._limit_inf = limit_inf
        # Atributos de rotação
        self._angle = angle
        self._angular_speed = angular_speed
        self._cos = math.cos(self._angle)
        self._sin = math.sin(self._angle) 
        # Atributos de escala
        self._scale = self._scale_x = self._scale_y = self._scale_z = scale
        self._scale_speed = scale_speed

    # Altera valores para desenho
    def set_identification(self, inicial_vert, num_vert):
        self._inicial_vert = inicial_vert
        self._num_vert = num_vert

    def _calc_limits_border(self, pos):
        if pos >= self._limit_sup:
            return self._limit_sup
        elif pos <= self._limit_inf:
            return self._limit_inf
        else:
            return pos
        
    def move_right(self):
        self._pos_x += self._linear_speed
        self._pos_x = self._calc_limits_border(self._pos_x)
    
    def move_left(self):
        self._pos_x -= self._linear_speed
        self._pos_x = self._calc_limits_border(self._pos_x)

    def move_up(self):
        self._pos_y += self._linear_speed
        self._pos_y = self._calc_limits_border(self._pos_y)
    
    def move_down(self):
        self._pos_y -= self._linear_speed
        self._pos_y = self._calc_limits_border(self._pos_y)

    def rotate_left(self):
        self._angle -= self._angular_speed
        self._cos = math.cos(self._angle)
        self._sin = math.sin(self._angle) 

    def rotate_right(self):
        self._angle += self._angular_speed
        self._cos = math.cos(self._angle)
        self._sin = math.sin(self._angle) 

    def scale_down(self):
        self._scale -= self._scale_speed
        self._scale_x = self._scale_y = self._scale_z = self._scale

    def scale_up(self):
        self._scale += self._scale_speed
        self._scale_x = self._scale_y = self._scale_z = self._scale
    
    def _mat_translation(self):
        return np.array([   1.0, 0.0, 0.0, self._pos_x, 
                            0.0, 1.0, 0.0, self._pos_y, 
                            0.0, 0.0, 1.0, self._pos_z, 
                            0.0, 0.0, 0.0,        1.0], np.float32)
    
    def _mat_scale(self):
        return np.array([   self._scale_x,           0.0,           0.0, 0.0, 
                                      0.0, self._scale_y,           0.0, 0.0, 
                                      0.0,           0.0, self._scale_z, 0.0, 
                                      0.0,           0.0,           0.0, 1.0], np.float32)
    
    def _rotation_z(self):  
        return np.array([   self._cos, -self._sin, 0.0, 0.0, 
                            self._sin,  self._cos, 0.0, 0.0, 
                                  0.0,        0.0, 1.0, 0.0, 
                                  0.0,        0.0, 0.0, 1.0], np.float32)
    
    def _rotation_x(self): 
        return np.array([   1.0,       0.0,        0.0, 0.0, 
                            0.0, self._cos, -self._sin, 0.0, 
                            0.0, self._sin,  self._cos, 0.0, 
                            0.0,       0.0,        0.0, 1.0], np.float32)
    
    def _rotation_y(self):
        return np.array([    self._cos,  0.0, self._sin, 0.0, 
                                   0.0,  1.0,       0.0, 0.0, 
                            -self._sin,  0.0, self._cos, 0.0, 
                                   0.0,  0.0,       0.0, 1.0], np.float32)
    
    def draw(self, loc_mat_rot_x, loc_mat_rot_y, loc_mat_rot_z, loc__mat_scale, loc_mat_transl, gl_Draw = GL_TRUE):
        glUniformMatrix4fv(loc_mat_rot_x, 1, gl_Draw, self._rotation_x()) 
        glUniformMatrix4fv(loc_mat_rot_y, 1, gl_Draw, self._rotation_y()) 
        glUniformMatrix4fv(loc_mat_rot_z, 1, gl_Draw, self._rotation_z()) 
        glUniformMatrix4fv(loc__mat_scale, 1, gl_Draw, self._mat_scale()) 
        glUniformMatrix4fv(loc_mat_transl, 1, gl_Draw, self._mat_translation()) 
        glDrawArrays(GL_TRIANGLES, self._inicial_vert, self._num_vert)

In [322]:
# Classe para criação de objetos com textura importados
class obj_wave(graphic_element):
    def __init__(   self, 
                    id_texture, path_obj, path_jpg,
                    inicial_vert = 0, num_vert = 0,
                    pos_x = 0, pos_y = 0, pos_z = 0,
                    angle = 0,
                    scale = 1,
                    linear_speed = 0.05, angular_speed = 0.139, scale_speed = 0.2,
                    limit_sup = 1,
                    limit_inf = -1,
                    ):
        # Inicializa atributos do pai
        super().__init__(   inicial_vert = inicial_vert, 
                            num_vert = num_vert,
                            pos_x = pos_x, pos_y = pos_y, pos_z = pos_z,
                            angle = angle,
                            scale = scale,
                            linear_speed = linear_speed, angular_speed = angular_speed, scale_speed = scale_speed,
                            limit_sup = limit_sup,
                            limit_inf = limit_inf
                        )
        # Atributos de identificação
        self._id_texture = id_texture
        self._path_obj = path_obj
        self._path_jpg = path_jpg
        # Gerando modelos e carregando texturas
        self._model = self._load_model_from_file()
        self._load_texture_from_file()

    # Função: carrega o arquivo Wavefront
    # Entrada: nome do arquivo
    # Saida: estrutura que armazena o elemento (vertices, textura e faces)
    def _load_model_from_file(self):
        vertices = []
        texture_coords = []
        faces = []

        material = None

        # Abre o arquivo obj (wavefront) para leitura
        for line in open(self._path_obj, "r"): ## para cada linha do arquivo .obj
            # Se for comentário, ignore esta linha e use a próxima
            if line.startswith('#'): continue

            # Quebra a linha por espaço
            values = line.split()
            # Se não há informações na linha, ignore esta linha e use a próxima
            if not values: continue

            # Recupera as informações
            ### Armazena coordenadas dos vertices do elemento no vetor vertices
            if values[0] == 'v':
                vertices.append(values[1:4])
            ### Armazena coordenadas das texturas no vetor texture_coords
            elif values[0] == 'vt':
                texture_coords.append(values[1:3])
            ### Define o material 
            elif values[0] in ('usemtl', 'usemat'):
                material = values[1]
            ### Armazena informações sobre a construção das faces
            elif values[0] == 'f':
                face = []
                face_texture = []
                # Para cada elemento da linha que define a função
                for bloco in values[1:]:
                    # Separa o elemento em vetor de elementos separando os números que são separados por /
                    positions = bloco.split('/')
                    # Adiciona o primeiro número na face (que representa o número da linha que encontra-se um vértice para da figura)
                    face.append(int(positions[0]))
                    # Se o vetor com elementos separados por / for maior ou igual que dois
                    # Se o segundo número do elemento for maior do que zero
                    if len(positions) >= 2 and len(positions[1]) > 0:
                        # Adicione o segundo número na face de textura (que representa o número da linha que encontra-se um vértice de textura da figura)
                        face_texture.append(int(positions[1]))
                    else:
                        # Se não for maior ou igual a dois ou não for maior que zero, coloque zero na textura
                        face_texture.append(0)
                # Após conseguir, provavelmente, os três valores para face, os três valores para textura e o tipo de material, insira na faces
                faces.append((face, face_texture, material))

        model = {}
        model['vertices'] = vertices
        model['texture'] = texture_coords
        model['faces'] = faces

        return model
    
    # Função: associa id com a textura
    # Entradas: o id que queremos associar e o caminho do arquivo .jpg
    # Saida: não possui, apenas associa
    def _load_texture_from_file(self):
        #Definindo o id
        glBindTexture(GL_TEXTURE_2D, self._id_texture)
        #Alterando configurações paramétricas de textura
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
        #Abre imagem
        img = Image.open(self._path_jpg)
        #Captura as dimensões
        img_width = img.size[0]
        img_height = img.size[1]
        #Transforma imagem para um sequência de bytes em formato raw de arquivo
        image_data = img.tobytes("raw", "RGB", 0, -1)
        #Carregando os dados da imagem
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

    # Função: retorna uma lista para coordenadas dos vertices do objeto e outra para as da textura
    # Como as listas de vértices só tem funcionalidade no momento em que são enviados para a GPU,
        # não há necessidade de manter dentro da classe
    def get_vertices_textures(self):
        vertices_list = []    
        textures_coord_list = []
        # Para cada um das faces (num_line(v), num_line(vt), material)
        for face in self._model['faces']:
            # Para cada um dos números que representa a linha do vértice
            for vertice_id in face[0]: # Pega o valor a coordenada do vértice
                vertices_list.append( self._model['vertices'][vertice_id-1] )
            # Para cada um dos números que representa a linha da coordenada da textura
            for texture_id in face[1]:  # Pega o valor a coordenada da textura
                textures_coord_list.append( self._model['texture'][texture_id-1] )
        return vertices_list, textures_coord_list
    
    #Overriding (sobreposição de função) - é necessário manter os mesmos parametros
    def draw(self, loc_mat_rot_x, loc_mat_rot_y, loc_mat_rot_z, loc_mat_scale, loc_mat_transl, gl_Draw = GL_TRUE):
        glUniformMatrix4fv(loc_mat_rot_x, 1, gl_Draw, self._rotation_x()) 
        glUniformMatrix4fv(loc_mat_rot_y, 1, gl_Draw, self._rotation_y()) 
        glUniformMatrix4fv(loc_mat_rot_z, 1, gl_Draw, self._rotation_z()) 
        glUniformMatrix4fv(loc_mat_scale, 1, gl_Draw, self._mat_scale()) 
        glUniformMatrix4fv(loc_mat_transl, 1, gl_Draw, self._mat_translation())

        # Ativa textura com id
        glBindTexture(GL_TEXTURE_2D, self._id_texture)
        # Desenha o elemento
        glDrawArrays(GL_TRIANGLES, self._inicial_vert, self._num_vert)

### Carregando os modelos

In [323]:
#Ativando texturas 2D
glEnable(GL_TEXTURE_2D)
#Gerando ids
num_textures = 10
textures = glGenTextures(num_textures)

#Carregando modelos
PATH_WAVE : str = 'objetos_wavefront'
#Carregando modelo de caixa
CAIXA_PATH_OBJ : str = f'{PATH_WAVE}\caixa\caixa.obj'
CAIXA_PATH_JPG : str = f'{PATH_WAVE}\caixa\caixa.jpg'
ID_CAIXA : int = 0
caixa = obj_wave(ID_CAIXA, CAIXA_PATH_OBJ, CAIXA_PATH_JPG, pos_x = -0.5, scale = 0.25, angle = 2)
#Carregando modelo de monstro
MONSTRO_PATH_OBJ : str = f'{PATH_WAVE}\monstro\monstro.obj'
MONSTRO_PATH_JPG : str = f'{PATH_WAVE}\monstro\monstro.jpg'
ID_MONSTRO : int = 1
monstro = obj_wave(ID_MONSTRO, MONSTRO_PATH_OBJ, MONSTRO_PATH_JPG, pos_x = 0.5, pos_y = -0.5, scale = 0.10)
#Carregando modelo de skybox
SKYBOX_PATH_OBJ : str = f'{PATH_WAVE}\skybox\skybox.obj'
SKYBOX_PATH_JPG : str = f'{PATH_WAVE}\skybox\skybox.jpg'
ID_SKYBOX : int = 2
skybox = obj_wave(ID_SKYBOX, SKYBOX_PATH_OBJ, SKYBOX_PATH_JPG)
#Lista de objetos
objs_wave = [caixa,monstro,skybox]

### Construção dos vetores de dados

In [324]:
total_len_pos = 0
total_len_text = 0
vertices_list_total = []
vertices_coord_total = []
identification = []

for obj in objs_wave:
    vertices_list, textures_coord_list = obj.get_vertices_textures()
    identification.append((total_len_pos, len(vertices_list)))
    total_len_pos += len(vertices_list)
    total_len_text += len(textures_coord_list)
    vertices_list_total += vertices_list
    vertices_coord_total += textures_coord_list

#Finaliza a modelagem dos dados de vértices
vertices = np.zeros(total_len_pos, [("position", np.float32, 3)])
vertices['position'] = vertices_list_total

#Finaliza a modelagem dos dados de texturas
textures = np.zeros(total_len_text, [("position", np.float32, 2)])
textures['position'] = vertices_coord_total

#Setando as identificações de desenho do objeto
for i, obj in enumerate(objs_wave):
    obj.set_identification(identification[i][0], identification[i][1])

### Manipulação dos espaços de dados

In [325]:
#Solicita dois buffers para GPU
buffer = glGenBuffers(2)

# Enviando dados de vértice
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

# Enviando dados de textura
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
#Subindo os dados de textura para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
#Encontrando informações de stride e offset das texturas
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "texture_coord" e habilitando
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
#Linkando dados ao atributo "texture_coord"
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

#Capturando posição dos uniform "mat_rot_x, mat_rot_y, mat_rot_z, mat_scale e mat_transl"
loc_mat_rot_x = glGetUniformLocation(program, "mat_rot_x")
loc_mat_rot_y = glGetUniformLocation(program, "mat_rot_y")
loc_mat_rot_z = glGetUniformLocation(program, "mat_rot_z")
loc_mat_scale = glGetUniformLocation(program, "mat_scale")
loc_mat_transl = glGetUniformLocation(program, "mat_transl")

### Eventos de teclado e mouse

In [326]:
polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global caixa, monstro, skybox

    if mods == 1:
        obj = monstro
    else:
        obj = skybox

    if key == 81: obj.rotate_left()   #Tecla Q
    if key == 82: obj.rotate_right()  #Tecla R

    if key == 70: obj.scale_up()      #Tecla F
    if key == 71: obj.scale_down()    #Tecla G

    if key == 87: obj.move_up()       #Tecla W
    if key == 83: obj.move_down()     #Tecla S
    if key == 65: obj.move_left()     #Tecla A
    if key == 68: obj.move_right()    #Tecla D

    # print('[key event] key=',key)
    # print('[key event] scancode=',scancode)
    # print('[key event] action=',action)
    # print('[key event] mods=',mods)
    # print('-------')
    
glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
    
glfw.set_mouse_button_callback(window,mouse_event)

### Exibindo na tela

In [327]:
glfw.show_window(window)

## Loop principal

#por que em antes estavamos trabalhdno com gl_false e agr n masi


#dúvida porque nao estou conseguindo mover minha caixa para os lados

In [328]:
# Habilita 3D
glEnable(GL_DEPTH_TEST)

while not glfw.window_should_close(window):
    # Lê eventos
    glfw.poll_events()
    # Limpa buffer de cor e Z-buffer
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    # Define cor como sendo RGB(1.0,1.0,1.0)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    # Define se será mostrado a malha ou não
    if polygonal_mode:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    else:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    
    caixa.draw(loc_mat_rot_x, loc_mat_rot_y, loc_mat_rot_z, loc_mat_scale, loc_mat_transl)
    monstro.draw(loc_mat_rot_x, loc_mat_rot_y, loc_mat_rot_z, loc_mat_scale, loc_mat_transl)
    skybox.draw(loc_mat_rot_x, loc_mat_rot_y, loc_mat_rot_z, loc_mat_scale, loc_mat_transl)
    # desenha_terreno()
    # desenha_casa()
    # desenha_monstro(rotacao_inc)  
 
    glfw.swap_buffers(window)

glfw.terminate()